In [ ]:
import os, datetime as dt
from pathlib import Path
import pandas as pd
import numpy as np
import requests
from dotenv import load_dotenv

# Load secrets
load_dotenv()
API_KEY = os.getenv("ALPHAVANTAGE_API_KEY")
TICKER  = os.getenv("TICKER", "AAPL")
assert API_KEY, "Please set ALPHAVANTAGE_API_KEY in your .env"

# Exact window to match your prices file
START_DATE = dt.date(2024, 8, 16)
END_DATE   = dt.date(2025, 8, 15)

# Find project root (folder that contains data/processed)
def find_project_root() -> Path:
    here = Path.cwd().resolve()
    for cand in [here, *here.parents]:
        if (cand / "data" / "processed").exists():
            return cand
    raise FileNotFoundError("Could not locate a 'data/processed' directory above current folder.")

ROOT = Path("..").resolve()
RAW  = ROOT / "data" / "raw";       RAW.mkdir(parents=True, exist_ok=True)
PROC = ROOT / "data" / "processed"; PROC.mkdir(parents=True, exist_ok=True)

def ts():
    return dt.datetime.now().strftime("%Y%m%d-%H%M")


Project root: /Users/brian/bootcamp_Brian_Chang/project
Raw data:     /Users/brian/bootcamp_Brian_Chang/project/data/raw
Processed:    /Users/brian/bootcamp_Brian_Chang/project/data/processed


In [2]:
def month_iter(start_date: dt.date, end_date: dt.date):
    """Yield (month_start, month_end) for each month overlapping [start_date, end_date]."""
    cur = dt.date(start_date.year, start_date.month, 1)
    while cur <= end_date:
        if cur.month == 12:
            nxt = dt.date(cur.year + 1, 1, 1)
        else:
            nxt = dt.date(cur.year, cur.month + 1, 1)
        m_start, m_end = cur, min(end_date, nxt - dt.timedelta(days=1))
        # clip the first month to start_date
        if m_start < start_date:
            m_start = start_date
        yield m_start, m_end
        cur = nxt

def fetch_alpha_news_sentiment_window(ticker: str, apikey: str, start: dt.date, end: dt.date, limit: int = 2000) -> pd.DataFrame:
    """
    Fetch Alpha Vantage NEWS_SENTIMENT for a given [start,end] window (UTC calendar).
    Returns article-level DataFrame with raw fields.
    """
    url = "https://www.alphavantage.co/query"
    # Alpha Vantage expects YYYYMMDDThhmm (UTC)
    time_from = dt.datetime(start.year, start.month, start.day, 0, 0)
    time_to   = dt.datetime(end.year,   end.month,   end.day,   23, 59)
    params = {
        "function": "NEWS_SENTIMENT",
        "tickers": ticker,
        "time_from": time_from.strftime("%Y%m%dT%H%M"),
        "time_to":   time_to .strftime("%Y%m%dT%H%M"),
        "limit": str(limit),
        "apikey": apikey,
        "sort": "LATEST",
    }
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    data = r.json()

    feed = data.get("feed", [])
    if not feed:
        return pd.DataFrame()

    rows = []
    for item in feed:
        overall_score = item.get("overall_sentiment_score")
        overall_label = item.get("overall_sentiment_label")

        # Pull ticker-specific sentiment (if present) for our ticker
        tick_info = {"ticker": ticker, "relevance_score": None,
                     "ticker_sentiment_score": None, "ticker_sentiment_label": None}
        for t in item.get("ticker_sentiment", []):
            if t.get("ticker") == ticker:
                tick_info = {
                    "ticker": t.get("ticker"),
                    "relevance_score": float(t.get("relevance_score", 0.0)),
                    "ticker_sentiment_score": float(t.get("ticker_sentiment_score", 0.0)),
                    "ticker_sentiment_label": t.get("ticker_sentiment_label"),
                }
                break

        rows.append({
            "time_published": item.get("time_published"),  # "YYYYMMDDThhmm" or "...hhmmss"
            "source": item.get("source"),
            "title": item.get("title"),
            "url": item.get("url"),
            "overall_sentiment_score": float(overall_score) if overall_score is not None else None,
            "overall_sentiment_label": overall_label,
            **tick_info
        })

    df = pd.DataFrame(rows)

    # Robust timestamp parsing (seconds first, then minutes, then free parser)
    pu = pd.to_datetime(df["time_published"], format="%Y%m%dT%H%M%S", errors="coerce", utc=True)
    m = pu.isna()
    if m.any():
        pu2 = pd.to_datetime(df.loc[m, "time_published"], format="%Y%m%dT%H%M", errors="coerce", utc=True)
        pu.loc[m] = pu2
    m2 = pu.isna()
    if m2.any():
        pu3 = pd.to_datetime(df.loc[m2, "time_published"], errors="coerce", utc=True)
        pu.loc[m2] = pu3

    df["published_utc"] = pu
    try:
        df["published_ny"] = df["published_utc"].dt.tz_convert("America/New_York")
    except Exception:
        df["published_ny"] = df["published_utc"].dt.tz_convert("US/Eastern")
    df["date_ny"]  = df["published_ny"].dt.date
    df["date_utc"] = df["published_utc"].dt.date

    # Coerce numeric
    for col in ["relevance_score","ticker_sentiment_score","overall_sentiment_score"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    return df


In [3]:
import datetime as dt, requests, pandas as pd, os
from dotenv import load_dotenv
load_dotenv()
API_KEY = os.getenv("ALPHAVANTAGE_API_KEY")
TICKER  = os.getenv("TICKER", "AAPL")
assert API_KEY, "Set ALPHAVANTAGE_API_KEY in .env"

def debug_window(ticker, start_date: dt.date, end_date: dt.date):
    url = "https://www.alphavantage.co/query"
    time_from = dt.datetime(start_date.year, start_date.month, start_date.day, 0, 0)
    time_to   = dt.datetime(end_date.year,   end_date.month,   end_date.day,   23, 59)
    params = {
        "function": "NEWS_SENTIMENT",
        "tickers": ticker,
        "time_from": time_from.strftime("%Y%m%dT%H%M"),
        "time_to":   time_to.strftime("%Y%m%dT%H%M"),
        "limit": "200",
        "apikey": API_KEY,
        "sort": "LATEST",
    }
    r = requests.get(url, params=params, timeout=30)
    data = r.json()
    print("HTTP", r.status_code)
    print("Keys present:", list(data.keys()))
    if "Note" in data:
        print("NOTE from API:", data["Note"])
    if "Information" in data:
        print("INFO from API:", data["Information"])
    feed = data.get("feed", [])
    print(f"Feed articles: {len(feed)}")
    if feed:
        print("First item fields:", list(feed[0].keys()))
        print("Sample time_published:", feed[0].get("time_published"))
    return data

# Try a 2-week window near the middle of your range:
_ = debug_window(TICKER, dt.date(2025, 3, 1), dt.date(2025, 3, 14))


HTTP 200
Keys present: ['items', 'sentiment_score_definition', 'relevance_score_definition', 'feed']
Feed articles: 50
First item fields: ['title', 'url', 'time_published', 'authors', 'summary', 'banner_image', 'source', 'category_within_source', 'source_domain', 'topics', 'overall_sentiment_score', 'overall_sentiment_label', 'ticker_sentiment']
Sample time_published: 20250314T214510


In [4]:
frames = []
for m_start, m_end in month_iter(START_DATE, END_DATE):
    dfm = fetch_alpha_news_sentiment_window(TICKER, API_KEY, m_start, m_end, limit=200)
    if not dfm.empty:
        frames.append(dfm)

articles = (pd.concat(frames, ignore_index=True)
            if frames else
            pd.DataFrame(columns=[
                "time_published","source","title","url","overall_sentiment_score","overall_sentiment_label",
                "ticker","relevance_score","ticker_sentiment_score","ticker_sentiment_label",
                "published_utc","published_ny","date_ny","date_utc"
            ]))

# Drop duplicates by (url, time_published)
if not articles.empty:
    articles = articles.drop_duplicates(subset=["url","time_published"]).reset_index(drop=True)

# Clip to window by NY date (safety)
if not articles.empty:
    mask = (articles["date_ny"] >= START_DATE) & (articles["date_ny"] <= END_DATE)
    articles = articles.loc[mask].reset_index(drop=True)

print("Articles fetched (deduped):", len(articles))
if not articles.empty:
    print("NY date range:", articles["date_ny"].min(), "→", articles["date_ny"].max())

articles.head(3)


Articles fetched (deduped): 650
NY date range: 2024-08-28 → 2025-08-15


,time_published,source,title,url,overall_sentiment_score,overall_sentiment_label,ticker,relevance_score,ticker_sentiment_score,ticker_sentiment_label,published_utc,published_ny,date_ny,date_utc
0,20240831T102400,Motley Fool,"Warren Buffett Sold Apple Stock, but These Bil...",https://www.fool.com/investing/2024/08/31/warr...,0.298569,Somewhat-Bullish,AAPL,0.731795,0.449680,Bullish,2024-08-31 10:24:00+00:00,2024-08-31 06:24:00-04:00,2024-08-31,2024-08-31
1,20240831T101000,Motley Fool,Will TSMC Be Worth More Than Apple by 2030?,https://www.fool.com/investing/2024/08/31/will...,0.258679,Somewhat-Bullish,AAPL,0.449723,0.348895,Somewhat-Bullish,2024-08-31 10:10:00+00:00,2024-08-31 06:10:00-04:00,2024-08-31,2024-08-31
2,20240831T091500,Motley Fool,3 Incredible FAANG Stocks You'll Want to Consi...,https://www.fool.com/investing/2024/08/31/incr...,0.265451,Somewhat-Bullish,AAPL,0.303592,0.254529,Somewhat-Bullish,2024-08-31 09:15:00+00:00,2024-08-31 05:15:00-04:00,2024-08-31,2024-08-31


In [8]:
raw_path = RAW / f"sentiment_alpha_articles_{TICKER}_{ts()}.csv"
articles.to_csv(raw_path, index=False)
print("Saved raw articles →", raw_path)

Saved raw articles → /Users/brian/bootcamp_Brian_Chang/project/data/raw/sentiment_alpha_articles_AAPL_20250823-1041.csv


In [17]:
def aggregate_daily_sentiment_strong(df: pd.DataFrame) -> pd.DataFrame:
    """
    Aggregate article-level sentiment to daily per ticker (NY market date).
    - Use ticker_sentiment_score where available; else fall back to overall_sentiment_score.
    - Use relevance_score as weight; else weight=1.
    - Keep daily rows sorted and tidy.
    """
    if df.empty:
        return pd.DataFrame(columns=["ticker","date","articles","sent_weighted","sent_mean","sent_overall_mean"])

    g = df.copy()
    if "ticker" not in g.columns:
        g["ticker"] = TICKER

    # Fallback score and weight
    g["score"] = g["ticker_sentiment_score"].where(g["ticker_sentiment_score"].notna(),
                                                   g["overall_sentiment_score"])
    g["w"]     = g["relevance_score"].fillna(1.0)
    g["wscore"]= g["w"] * g["score"]

    # Require a usable date key (NY dates preferred)
    date_key = "date_ny" if "date_ny" in g.columns and g["date_ny"].notna().any() else "date_utc"
    g = g[~g[date_key].isna()].copy()

    daily = (g.groupby(["ticker", date_key])
               .agg(
                   articles=("score","size"),
                   wscore_sum=("wscore","sum"),
                   w_sum=("w","sum"),
                   sent_mean=("score","mean"),
                   sent_overall_mean=("overall_sentiment_score","mean"),
               )
               .reset_index())

    daily["sent_weighted"] = np.where(daily["w_sum"] > 0, daily["wscore_sum"]/daily["w_sum"], np.nan)
    daily = (daily
             .rename(columns={date_key: "date"})
             .drop(columns=["wscore_sum","w_sum"])
             .sort_values("date")
             .reset_index(drop=True))
    return daily

daily = aggregate_daily_sentiment_strong(articles)
print("Daily rows:", len(daily))
daily.head(10)


Daily rows: 53


,ticker,date,articles,sent_mean,sent_overall_mean,sent_weighted
0,AAPL,2024-08-28,13,0.131668,0.207986,0.188072
1,AAPL,2024-08-29,13,0.162060,0.230984,0.225318
2,AAPL,2024-08-30,18,0.188095,0.232715,0.277089
3,AAPL,2024-08-31,6,0.348782,0.244972,0.357171
4,AAPL,2024-09-26,6,0.178697,0.274282,0.228993
5,AAPL,2024-09-27,13,0.136303,0.220684,0.170078
6,AAPL,2024-09-28,11,0.196241,0.215400,0.294228
7,AAPL,2024-09-29,4,0.383651,0.288153,0.386503
8,AAPL,2024-09-30,16,0.128645,0.192185,0.161520
9,AAPL,2024-10-29,12,0.124309,0.232364,0.258968


In [20]:
# Load latest processed prices
price_candidates = sorted(PROC.glob("prices_preprocessed_*.csv"), key=lambda p: p.stat().st_mtime, reverse=True)
if not price_candidates:
    price_candidates = sorted(PROC.glob("prices_with_features*.csv"), key=lambda p: p.stat().st_mtime, reverse=True)
assert price_candidates, "No processed prices CSV found in data/processed/"
prices_path = price_candidates[0]

dfp = pd.read_csv(prices_path)
dfp["date"] = pd.to_datetime(dfp["date"], errors="coerce", utc=True)
try:
    dfp["date_ny"] = dfp["date"].dt.tz_convert("America/New_York").dt.date
except Exception:
    dfp["date_ny"] = dfp["date"].dt.tz_convert("US/Eastern").dt.date

# Single-ticker project safeguard
if "ticker" not in dfp.columns:
    dfp["ticker"] = TICKER

# Trading-day calendar
trading_days = (dfp.loc[(dfp["date_ny"] >= START_DATE) & (dfp["date_ny"] <= END_DATE), ["ticker","date_ny"]]
                  .dropna().drop_duplicates()
                  .rename(columns={"date_ny":"date"})
                  .sort_values("date"))

# Join sentiment to trading days
sent_aligned = (trading_days.merge(daily, on=["ticker","date"], how="left")
                .sort_values("date").reset_index(drop=True))

# Fill policies
sent_aligned["sent_neutral0"] = sent_aligned["sent_weighted"].fillna(0.0)            # neutral
sent_aligned["sent_ffill2"]   = sent_aligned["sent_weighted"].ffill(limit=2).fillna(0.0)  # short persistence

print("Trading days:", len(trading_days), "| with any news:", sent_aligned["sent_weighted"].notna().sum())


Trading days: 250 | with any news: 41


In [21]:
aligned_path = PROC / f"sentiment_daily_aligned_{TICKER}_{ts()}.csv"
sent_aligned.to_csv(aligned_path, index=False)
print("Saved aligned sentiment →", aligned_path)

Saved aligned sentiment → /Users/brian/bootcamp_Brian_Chang/project/data/processed/sentiment_daily_aligned_AAPL_20250823-1109.csv
